In [1]:
import requests
from bs4 import BeautifulSoup as soup
import spacy
import json
import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_sm')
from string import punctuation
import numbers

def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    page = soup(req.text, parser)
    return json.loads("".join(page.find("script", {"type":"application/ld+json"}).contents))

In [2]:
listofdrinks = [
"https://www.allrecipes.com/recipe/138002/classic-whiskey-sour/?internalSource=hub%20recipe&referringContentType=Search",
"https://www.allrecipes.com/recipe/217233/whiskey-ginger-cocktail/?internalSource=hub%20recipe&referringContentType=Search",
"https://www.allrecipes.com/recipe/222415/manhattan-cocktail/?internalSource=hub%20recipe&referringContentType=Search",
"https://www.allrecipes.com/recipe/222414/gimlet-cocktail/?internalSource=hub%20recipe&referringContentType=Search",
"https://www.allrecipes.com/recipe/222511/tom-collins-cocktail/?internalSource=hub%20recipe&referringContentType=Search",
"https://www.allrecipes.com/recipe/221305/aviation-cocktail/?internalSource=hub%20recipe&referringContentType=Search",
"https://www.allrecipes.com/recipe/221894/singapore-sling-cocktail/?internalSource=recipe%20hub&referringContentType=Search&clickId=cardslot%2021",
"https://www.allrecipes.com/recipe/221310/moscow-mule-cocktail/",
"https://www.allrecipes.com/recipe/221891/cosmopolitan-cocktail/",
"https://www.allrecipes.com/recipe/221893/vodka-martini-cocktail/",
"https://www.allrecipes.com/recipe/49444/mint-juleps/"
]

In [3]:
def get_name(jsonld):
    try:
        name = jsonld['name']
    except:
        name = jsonld[1]['name']
    
    lst = []
    for i, each in enumerate(name.lower().split()):
        lst.append(each.capitalize())
    name = ''.join(lst)
    
    return name

In [4]:
def get_ingredients_bonapp(jsonld):
    try:
        ingredients = jsonld['recipeIngredient']
    except:
        ingredients = jsonld[1]["recipeIngredient"]

    core = []
    for item in ingredients:
        if item.split()[0].isnumeric():
            doc = nlp(' '.join(item.split()[2:]))
        else:
            doc = nlp(item)
#         doc = nlp(item)
#         displacy.render(doc, style="dep")
        key = []
        for token in doc:
            print(token, token.dep_, token.pos_)
            if token.dep_ == "compound" or token.dep_ == "ROOT":
                key.append(token.text)
    #             print(token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop, token.children, token.head)
    #             displacy.render(doc, style="dep")
        core.append(' '.join(key))
        
    for j, item in enumerate(core):
        if item == 'syrup':
            core[j] = 'simpleSyrup'
            continue
        elif len(item.split()) == 1:
            item = item.lower()
        else:
            lst = []
            for i, each in enumerate(item.lower().split()):
                if i == 0:
                    lst.append(each)
                else:
                    lst.append(each.capitalize())
            item = ''.join(lst)
        core[j] = item

    
    return core

In [5]:
def get_ingredients_allrecipes(jsonld):
    try:
        ingredients = jsonld['recipeIngredient']
    except:
        ingredients = jsonld[1]["recipeIngredient"]
        
    core = []
    measurement = ['ounce', 'ounces', 'cup', 'cups', 'tablespoon', 'tablespoons', 'fluid']
    
    for i, ingredient in enumerate(ingredients):
        lst = [ing.replace('-',' ') for ing in ingredient.split()]
        while lst[0].isnumeric() or any([x == lst[0] for x in measurement]):
            lst.pop(0)
        word = ' '.join(lst)
        
        doc = nlp(word)
        lst = []
        stop = set()
        for token in doc:
#             print(token, token.pos_, token.dep_)
#             if token.text == 'brandy':
#                 print(token.dep_, token.pos_)
#                 print(token.head)
#                 print(stop)
            if token.dep_ == 'nmod' or token.pos_ == "ADV" or token.pos_ == "VERB" or token.pos_ == "ADP" or token.head in stop:
                stop.add(token)
            else:
                lst.append(token.text)
#                 w = word.split()
#                 w.remove(token.text)
#                 word = ' '.join(w)
        final_word = ' '.join(lst)
    
#         compare = []
#         for token in nlp(final_word):
#             if token.head in stop:
#                 compare.append(token)
        
#         if final_word == ' '.join(compare):
#             final_word = word
    
        w = []
        for token in final_word.split():
            token = ''.join(e for e in token if e.isalnum())
            w.append(token)
            
        if 'juice' in w:
            v = w
            v.remove('juice')
            x = []
            for n in nlp(' '.join(v)).noun_chunks:
                x.append(n.text)
            w = x
            w.append('juice')
            
        final_word = ' '.join(w)
        
        
        if final_word.strip() not in core:
            core.append(final_word.strip())
            
    for j, item in enumerate(core):
        if len(item.split()) == 1:
            item = item.capitalize()
        else:
            lst = [e.capitalize() for e in item.lower().split()]
#             lst = []
#             for i, each in enumerate(item.lower().split()):
#                 lst.append(each.capitalize())
            item = ''.join(lst)
        core[j] = item
            
    return core
    
    

In [124]:
def get_ingredients_allrecipes_hardcode(jsonld):
    try:
        ingredients = jsonld['recipeIngredient']
    except:
        ingredients = jsonld[1]["recipeIngredient"]
        
    core = []
    garnish = []
    measurement = ['ounce', 'ounces', 'cup', 'cups', 'tablespoon', 'tablespoons', 'fluid', 'dash', 'sprig']
    descriptors = ['fresh ']
    liquors = ['whiskey', 'gin', 'vodka', 'tequila', 'rum', 'bourbon', 'brandy']
    
    print(ingredients)
    
    for i, ingredient in enumerate(ingredients):
        
#         ingredient = [i for i in ingredient if i.isalnum()]
#         ingredient = ''.join(ingredient)
        
        main = None
        if ',' in ingredient:
            ing = ingredient.split(',')
            for each in ing:
                if each[0].isnumeric():
                    main = each
            ingredient = main
            print(main)
        
        # Replacing quantifiers from the front of the sentence
        lst = [ing.replace('-',' ') for ing in ingredient.split()]
        while lst[0].isnumeric() or any([x == lst[0] for x in measurement]):
            lst.pop(0)
        word = ' '.join(lst)
        
        # Replacing descriptors from sentence
        for d in descriptors:
            word = word.replace(d, '')
            
        # Getting just simple syrup
#         if 'simple syrup' in word:
#             core.append('SimpleSyrup')
#             continue
            
        if 'syrup' in word:
            wrdlst = word.split()
            i = wrdlst.index('syrup')
            try:
                wrd = [wrdlst[i-1].capitalize(), wrdlst[i].capitalize()]
                wrd = ''.join(wrd)
                core.append(wrd)
            except:
                core.append("Syrup")
            continue
        
        # Getting juice
        if 'juice' in word:
            # Two ways to do this, one with juice, the other without
            nojuice = word.replace('juice', '')
            
#             juicelst = []
#             for token in nlp(word).noun_chunks:
#                 if token.root.head.text == 'juice':
#                     juicelst.append(token.text)
            
            nojuicelst = []
            for token in nlp(nojuice).noun_chunks:
                nojuicelst.append(token.text)
            nojuicelst = [x.capitalize() for x in nojuicelst]
            
            fruit = ' '.join(nojuicelst)
            
            core.append('(juiceOfFn %s)' % fruit)
            continue
        
        # Getting just ice
        if 'ice' in word or 'ice cube' in word:
            core.append('Ice')
            continue
            
        # Getting garnish
        if 'garnish' in word:
            word = word.replace('garnish','').replace('for', '')
            lst = [x.capitalize() for x in word.split()]
            core.append(''.join(lst))
            garnish.append(''.join(lst))
            continue
        
        # Getting liqueurs
        if 'liqueur' in word:
            liq = word.replace('liqueur','')
            liq = [x.capitalize() for x in liq.split()]
            l = ' '.join(liq)
            
            core.append('(liqueurOfFn %s)' % l)
            continue
            
        # Getting distilled spirits
        hasspirits = False
        for l in liquors:
            if l in word:
                liq = [x.capitalize() for x in word.split()]
                full = ''.join(liq)
                core.append(full)
                hasspirits = True
                break
        if hasspirits == True:
            continue
        
        ok = [x.capitalize() for x in word.split()]
        ok = ''.join(ok)
        core.append(ok)
        
        
#         doc = nlp(word)
#         lst = []
#         stop = set()
#         for token in doc:
# #             print(token, token.pos_, token.dep_)
# #             if token.text == 'brandy':
# #                 print(token.dep_, token.pos_)
# #                 print(token.head)
# #                 print(stop)
#             if token.dep_ == 'nmod' or token.pos_ == "ADV" or token.pos_ == "VERB" or token.pos_ == "ADP" or token.head in stop:
#                 stop.add(token)
#             else:
#                 lst.append(token.text)
# #                 w = word.split()
# #                 w.remove(token.text)
# #                 word = ' '.join(w)
#         final_word = ' '.join(lst)

    res = [] 
    [res.append(x) for x in core if x not in res] 
            
    return res, garnish
    
    

In [125]:
def into_krf(jsonld):
    ingredients, garnish = get_ingredients_allrecipes_hardcode(jsonld)
    name = get_name(jsonld)
    
    query = []
    for i in ingredients:
        query.append("(ingredientOf %s %s)" % (i, name))
    for g in garnish:
        query.append("(isGarnish %s)" % g)
        
    return query
    
    

In [132]:
def make_krf(listofdrinks):
    for i, url in enumerate(listofdrinks):
        jsonld = get_ld_json(url)
        n = get_name(jsonld)
        name = "(isa %s CocktailDrink)" % n
        q = into_krf(jsonld)
        
        cast = []
        cast.append(name)
        for item in q:
            cast.append(item)
            
        with open("../knowledge_base/%s.krf" % n, mode='w') as file:
            for line in cast:
                file.write(line + '\n')
        

In [134]:
make_krf(listofdrinks)

['1 fluid ounce simple syrup', '2 fluid ounces fresh lemon juice', '5 fluid ounces whiskey', 'ice cubes', '3 maraschino cherries for garnish']
['ice', '2 fluid ounces rye whiskey', '2 fluid ounces ginger ale, or to taste']
2 fluid ounces ginger ale
['2 fluid ounces rye whiskey', '½ fluid ounce sweet vermouth', '1 dash Angostura bitters', '1 cup ice cubes', '1 maraschino cherry']
['1\u2009½ fluid ounces gin', '1 fluid ounce lime juice', '1 teaspoon simple syrup', '1 cup ice', '1 lime wheel']
['1\u2009½ cups ice', '2 fluid ounces gin', '¾ fluid ounce lemon juice', '½ fluid ounce simple syrup', '1 cup ice', '2 fluid ounces club soda', '1 lemon wedge']
['2 fluid ounces gin', '1 tablespoon maraschino liqueur', '¼ fluid ounce lemon juice', '1 cup ice', 'maraschino cherry for garnish']
['1 cup ice', '1\u2009½ fluid ounces gin', '½ fluid ounce cherry-flavored brandy', '¼ fluid ounce triple sec', '¼ fluid ounce Benedictine® liqueur', '4 fluid ounces pineapple juice', '½ fluid ounce lime juice',

In [123]:
for i, url in enumerate(listofdrinks):
    jsonld = get_ld_json(url)
    # core = get_ingredients(jsonld)
#         core = get_ingredients_allrecipes_hardcode(jsonld)
    # core = get_ingredients_nounchunks(jsonld)
    name = get_name(jsonld)
    into_krf(jsonld)
    print('-----------------------------------------------')

['1 fluid ounce simple syrup', '2 fluid ounces fresh lemon juice', '5 fluid ounces whiskey', 'ice cubes', '3 maraschino cherries for garnish']
['(ingredientOf SimpleSyrup ClassicWhiskeySour)', '(ingredientOf (juiceOfFn Lemon) ClassicWhiskeySour)', '(ingredientOf Whiskey ClassicWhiskeySour)', '(ingredientOf Ice ClassicWhiskeySour)', '(ingredientOf MaraschinoCherries ClassicWhiskeySour)']
-----------------------------------------------
['ice', '2 fluid ounces rye whiskey', '2 fluid ounces ginger ale, or to taste']
2 fluid ounces ginger ale
['(ingredientOf Ice WhiskeyGingerCocktail)', '(ingredientOf RyeWhiskey WhiskeyGingerCocktail)', '(ingredientOf GingerAle WhiskeyGingerCocktail)']
-----------------------------------------------
['2 fluid ounces rye whiskey', '½ fluid ounce sweet vermouth', '1 dash Angostura bitters', '1 cup ice cubes', '1 maraschino cherry']
['(ingredientOf RyeWhiskey ManhattanCocktail)', '(ingredientOf SweetVermouth ManhattanCocktail)', '(ingredientOf AngosturaBitters

In [93]:
jsonld = get_ld_json("https://www.allrecipes.com/recipe/221305/aviation-cocktail/?internalSource=hub%20recipe&referringContentType=Search")
# core = get_ingredients(jsonld)
# core = get_ingredients_allrecipes_hardcode(jsonld)
# core = get_ingredients_nounchunks(jsonld)
name = get_name(jsonld)
into_krf(jsonld)

['(ingredientOf (liqueurOfFn Maraschino) AviationCocktail)', '(ingredientOf (juiceOfFn Lemon) AviationCocktail)', '(ingredientOf Ice AviationCocktail)', '(ingredientOf MaraschinoCherry AviationCocktail)']
['MaraschinoCherry']


In [69]:
doc = nlp('lime wedge for garnish')
for token in doc:
    print(token, token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop, token.children, token.head)
#     print(token.head, token.text)
displacy.render(doc, style="dep")

lime lime PROPN NNP compound xxxx True False <generator object at 0x000001BFC4558B80> wedge
wedge wedge PROPN NNP ROOT xxxx True False <generator object at 0x000001BFC4558B80> wedge
for for ADP IN prep xxx True True <generator object at 0x000001BFC4558B80> wedge
garnish garnish NOUN NN pobj xxxx True False <generator object at 0x000001BFC4558B80> for


In [84]:
doc = nlp("pink")
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)